## Fetch Data

In [0]:
!wget https://storage.googleapis.com/chestxray14/downsized/images_001.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_002.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_003.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_004.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_005.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_006.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_007.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_008.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_009.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_010.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_011.tar.gz 
!wget https://storage.googleapis.com/chestxray14/downsized/images_012.tar.gz

--2019-04-22 07:05:39--  https://storage.googleapis.com/chestxray14/downsized/images_001.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.176, 2607:f8b0:4004:803::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107261934 (102M) [application/x-gzip]
Saving to: ‘images_001.tar.gz’

images_001.tar.gz   100%[===================>] 102.29M   190MB/s    in 0.5s    

2019-04-22 07:05:40 (190 MB/s) - ‘images_001.tar.gz’ saved [107261934/107261934]

/content/images_01.tar.gz: Scheme missing.
FINISHED --2019-04-22 07:05:40--
Total wall clock time: 0.7s
Downloaded: 1 files, 102M in 0.5s (190 MB/s)
--2019-04-22 07:05:40--  https://storage.googleapis.com/chestxray14/downsized/images_002.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.208, 2607:f8b0:4004:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.

In [0]:
!rm -rf images
!tar -xzf images_001.tar.gz
!tar -xzf images_002.tar.gz
!tar -xzf images_003.tar.gz
!tar -xzf images_004.tar.gz
!tar -xzf images_005.tar.gz
!tar -xzf images_006.tar.gz
!tar -xzf images_007.tar.gz
!tar -xzf images_008.tar.gz
!tar -xzf images_009.tar.gz
!tar -xzf images_010.tar.gz
!tar -xzf images_011.tar.gz
!tar -xzf images_012.tar.gz

tar (child): images_002.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [0]:
!rm -rf *.gz

## Fetch Labels

In [1]:
!wget https://github.gatech.edu/raw/gist/rdesai65/e127e4cade5054eaadc8f886c7223e0a/raw/adeaaf5e4028007428b224266f820d43ec984e96/nih_labels.csv?token=AABESdCtB18y9CMnMHzcp38nd8WR1crdks5czoeRwA%3D%3D -O nih_labels.csv

--2019-04-28 07:06:33--  https://github.gatech.edu/raw/gist/rdesai65/e127e4cade5054eaadc8f886c7223e0a/raw/adeaaf5e4028007428b224266f820d43ec984e96/nih_labels.csv?token=AABESdCtB18y9CMnMHzcp38nd8WR1crdks5czoeRwA%3D%3D
Resolving github.gatech.edu (github.gatech.edu)... 130.207.175.93
Connecting to github.gatech.edu (github.gatech.edu)|130.207.175.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10817565 (10M) [text/plain]
Saving to: ‘nih_labels.csv’

nih_labels.csv      100%[===================>]  10.32M  38.2MB/s    in 0.3s    

2019-04-28 07:06:33 (38.2 MB/s) - ‘nih_labels.csv’ saved [10817565/10817565]



## ChexDataset

In [0]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os
from PIL import Image


class ChexDataset(Dataset):

    def __init__(
            self,
            path_to_images,
            path_to_labels_csv,
            fold,
            transform=None,
            sample=0,
            label="any",
            sampled_images_path=None):

        # temporary, if we figure out way to do this during pre-processing we dont need this
        self.transform = transform
        self.path_to_images = path_to_images
        self.df = pd.read_csv(path_to_labels_csv)
        self.df = self.df[self.df['fold'] == fold]  # filter all images belonging to this fold
        
        filenames = set([file for file in os.listdir('images') if file.endswith('.png')])
        self.df = self.df.loc[self.df['Image Index'].isin(filenames)].reset_index(drop=True)

        if sampled_images_path is not None:
            sampled_images = pd.read_csv(sampled_images_path)
            self.df = pd.merge(left=self.df, right=sampled_images, how="inner", on="Image Index")

        # can limit to sample, useful for testing
        # if fold == "train" or fold =="val": sample=500
        if 0 < sample < len(self.df):
            self.df = self.df.sample(sample)

        if not label == "any":  # can filter for positive findings of the kind described; useful for evaluation
            if label in self.df.columns:
                if len(self.df[self.df[label] == 1]) > 0:
                    self.df = self.df[self.df[label] == 1]
                else:
                    print("No positive cases exist for " + label + ", returning all unfiltered cases")
            else:
                print("cannot filter on label " + label +
                      " as not in data - please check spelling")

        self.df = self.df.set_index("Image Index")
        self.PRED_LABEL = [
            'Atelectasis',
            'Cardiomegaly',
            'Effusion',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pneumonia',
            'Pneumothorax',
            'Consolidation',
            'Edema',
            'Emphysema',
            'Fibrosis',
            'Pleural_Thickening',
            'Hernia']
        RESULT_PATH = "results/"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
#         print("getItem: "+ str(idx)+"\n")

        # read custom format
        # convert to image format:

        image = Image.open(
            os.path.join(
                self.path_to_images,
                self.df.index[idx]))
        image = image.convert('RGB')

        label = np.zeros(len(self.PRED_LABEL), dtype=int)
        for i in range(0, len(self.PRED_LABEL)):
            # can leave zero if zero, else make one
            if self.df[self.PRED_LABEL[i].strip()].iloc[idx].astype('int') > 0:
                label[i] = self.df[self.PRED_LABEL[i].strip()
                ].iloc[idx].astype('int')

        if self.transform:
            image = self.transform(image)

        return image, label, self.df.index[idx]




## EvaluateModel

In [0]:
import torch
import pandas as pd
# import ChexDataset as CXR
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import sklearn
import sklearn.metrics as sklm
from torch.autograd import Variable
import numpy as np


def make_pred_multilabel(data_transforms, model, PATH_TO_IMAGES, PATH_TO_LABELS, use_gpu=False,test_sampled=0):
    """
    Gives predictions for test fold and calculates AUCs using previously trained model
    Args:
        data_transforms: torchvision transforms to preprocess raw images; same as validation transforms
        model: densenet-121 from torchvision previously fine tuned to training data
        PATH_TO_IMAGES: path at which NIH images can be found
    Returns:
        pred_df: dataframe containing individual predictions and ground truth for each test image
        auc_df: dataframe containing aggregate AUCs by train/test tuples
    """

    # calc preds in batches of 16, can reduce if our GPU has less RAM
    BATCH_SIZE = 16

    # set model to eval mode; required for proper predictions given use of batchnorm
    model.train(False)

    # create data_loader
#     dataset = CXR.ChexDataset(
    dataset = ChexDataset(
        path_to_images=PATH_TO_IMAGES,
        path_to_labels_csv=PATH_TO_LABELS,
        fold="test",
        transform=data_transforms['val'],
        sample=test_sampled
    )
    data_loader = torch.utils.data.DataLoader(
        dataset, BATCH_SIZE, shuffle=False, num_workers=8)
    size = len(dataset)

    # create empty dfs
    pred_df = pd.DataFrame(columns=["Image Index"])
    true_df = pd.DataFrame(columns=["Image Index"])

    # iterate over data_loader
    for i, data in enumerate(data_loader):

        inputs, labels, _ = data
        if use_gpu:
          inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        true_labels = labels.cpu().data.numpy()
        batch_size = true_labels.shape

        outputs = model(inputs)
        probs = outputs.cpu().data.numpy()

        # get predictions and true values for each item in batch
        for j in range(0, batch_size[0]):
            thisrow = {}
            truerow = {}
            thisrow["Image Index"] = dataset.df.index[BATCH_SIZE * i + j]
            truerow["Image Index"] = dataset.df.index[BATCH_SIZE * i + j]

            # iterate over each entry in prediction vector; each corresponds to
            # individual label
            for k in range(len(dataset.PRED_LABEL)):
                thisrow["prob_" + dataset.PRED_LABEL[k]] = probs[j, k]
                truerow[dataset.PRED_LABEL[k]] = true_labels[j, k]

            pred_df = pred_df.append(thisrow, ignore_index=True)
            true_df = true_df.append(truerow, ignore_index=True)

        if (i % 10 == 0):
            print(str(i * BATCH_SIZE))

    auc_df = pd.DataFrame(columns=["label", "auc"])

    # calc AUCs
    for column in true_df:

        if column not in [
            'Atelectasis',
            'Cardiomegaly',
            'Effusion',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pneumonia',
            'Pneumothorax',
            'Consolidation',
            'Edema',
            'Emphysema',
            'Fibrosis',
            'Pleural_Thickening',
            'Hernia']:
            continue
        actual = true_df[column]
        pred = pred_df["prob_" + column]
        thisrow = {}
        thisrow['label'] = column
        thisrow['auc'] = np.nan
        try:
            thisrow['auc'] = sklm.roc_auc_score(
                actual.as_matrix().astype(int), pred.as_matrix())
        except BaseException:
            print("can't calculate auc for " + str(column))
        auc_df = auc_df.append(thisrow, ignore_index=True)

    pred_df.to_csv("results/preds.csv", index=False)
    auc_df.to_csv("results/aucs.csv", index=False)
    return pred_df, auc_df




## ChexModel

In [0]:
from __future__ import print_function, division

# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# image imports
from skimage import io, transform
from PIL import Image

# general imports
import os
import time
from shutil import copyfile
from shutil import rmtree

# data science imports
import pandas as pd
import numpy as np
import csv

# import ChexDataset as CXR
# import EvaluateModel as EM

use_gpu = torch.cuda.is_available()
gpu_count = torch.cuda.device_count()
print("Available GPU count:" + str(gpu_count))


def checkpoint(model, best_loss, epoch, LR):
    """
    Saves checkpoint of torchvision model during training.

    Args:
        model: torchvision model to be saved
        best_loss: best val loss achieved so far in training
        epoch: current epoch of training
        LR: current learning rate in training
    Returns:
        None
    """

    print('saving')
    state = {
        'model': model,
        'best_loss': best_loss,
        'epoch': epoch,
        'rng_state': torch.get_rng_state(),
        'LR': LR
    }

    torch.save(state, 'results/checkpoint')


def train_model(
        model,
        criterion,
        optimizer,
        learning_rate,
        num_epochs,
        data_loaders,
        dataset_sizes,
        weight_decay,
        use_gpu = False
):
    """
    Fine tunes torchvision model to NIH CXR data.

    Args:
        model: torchvision model to be finetuned (densenet-121 in this case)
        criterion: loss criterion (binary cross entropy loss, BCELoss)
        optimizer: optimizer to use in training (SGD)
        learning_rate: learning rate
        num_epochs: continue training up to this many epochs
        data_loaders: pytorch train and val dataloaders
        dataset_sizes: length of train and val datasets
        weight_decay: weight decay parameter we use in SGD with momentum
    Returns:
        model: trained torchvision model
        best_epoch: epoch on which best model val loss was obtained

    """
    since = time.time()

    start_epoch = 1
    best_loss = 999999
    best_epoch = -1
    last_train_loss = -1

    # iterate over epochs
    for epoch in range(start_epoch, num_epochs + 1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # set model to train or eval mode based on whether we are in train or
        # val; necessary to get correct predictions given batchnorm
        for phase in ['train', 'val']:
            print("Epoch {}/{}, phase:{}".format(epoch, num_epochs, phase))
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0

            i = 0
            total_done = 0
#             print("Epoch {}/{}, phase:{}, start".format(epoch, num_epochs, phase))
            # iterate over all data in train/val dataloader:
            for data in data_loaders[phase]:
#                 print("for:"+str(i))
                i += 1
                inputs, labels, _ = data
#                 print("for-2:")
                batch_size = inputs.shape[0]
#                 print("for-3:")
                labels = labels.float()
#                 print("Epoch {}/{}, phase:{}, got input,labels".format(epoch, num_epochs, phase))
                if use_gpu:
                  inputs = Variable(inputs.cuda())
                  labels = Variable(labels.cuda()).float()
                  
#                 print("Epoch {}/{}, phase:{}, start-model(inputs)".format(epoch, num_epochs, phase))                  
                outputs = model(inputs)
#                 print("Epoch {}/{}, phase:{}, end-model(inputs)".format(epoch, num_epochs, phase))

                # calculate gradient and update parameters in train phase
#                 print("Epoch {}/{}, phase:{}, update grad".format(epoch, num_epochs, phase))
                optimizer.zero_grad()
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
#                 print("Epoch {}/{}, phase:{}, running_loss: {}".format(epoch, num_epochs, phase, running_loss))
#                 current_loss = 
#                 if loss.data[0]
                running_loss += loss.item() * batch_size
                print("Epoch {}/{}, phase:{}, running_loss: {}, current_loss: {}".format(epoch, num_epochs, phase, running_loss, loss.item()))

            epoch_loss = running_loss / dataset_sizes[phase]

            if phase == 'train':
                last_train_loss = epoch_loss

            print(phase + ' epoch {}:loss {:.4f} with data size {}'.format(
                epoch, epoch_loss, dataset_sizes[phase]))

            # decay learning rate if no val loss improvement in this epoch
            if phase == 'val' and epoch_loss > best_loss:
                print("decay loss from " + str(learning_rate) + " to " +
                      str(learning_rate / 10) + " as not seeing improvement in val loss")
                learning_rate = learning_rate / 10
                # create new optimizer with lower learning rate
                optimizer = optim.SGD(
                    filter(
                        lambda p: p.requires_grad,
                        model.parameters()),
                    lr=learning_rate,
                    momentum=0.9,
                    weight_decay=weight_decay)
                print("created new optimizer with LR " + str(learning_rate))

            # checkpoint model if has best val loss yet
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                checkpoint(model, best_loss, epoch, learning_rate)

            # log training and validation loss over each epoch
            if phase == 'val':
                with open("results/log_train", 'a') as logfile:
                    logw_riter = csv.writer(logfile, delimiter=',')
                    if (epoch == 1):
                        logw_riter.writerow(["epoch", "train_loss", "val_loss"])
                    logw_riter.writerow([epoch, last_train_loss, epoch_loss])

        total_done += batch_size
        if total_done % (100 * batch_size) == 0:
            print("completed " + str(total_done) + " so far in epoch")

        # break if no val loss improvement in 3 epochs
        if (epoch - best_epoch) >= 3:
            print("no improvement in 3 epochs, break")
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights to return
    checkpoint_best = torch.load('results/checkpoint')
    model = checkpoint_best['model']

    return model, best_epoch


def train_cnn(PATH_TO_IMAGES, PATH_TO_LABELS, learning_rate, WEIGHT_DECAY, use_gpu=False,train_sampled=0,val_sampled=0,test_sampled=0, NUM_EPOCHS=8, BATCH_SIZE=16):
    """
    Train torchvision model to NIH data given high level hyperparameters.

    Args:
        PATH_TO_IMAGES: path to NIH images
        PATH_TO_LABELS: path to csv which contains labels
        learning_rate: learning rate
        WEIGHT_DECAY: weight decay parameter for SGD

    Returns:
        preds: torchvision model predictions on test fold with ground truth for comparison
        aucs: AUCs for each train,test tuple

    """
#     NUM_EPOCHS = 20
#     BATCH_SIZE = 16

    try:
        rmtree('results/')
    except BaseException:
        pass  # directory doesn't yet exist, no need to clear it
    os.makedirs("results/")

    # use imagenet mean,std for normalization
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    N_LABELS = 14  # we are predicting 14 labels

    # load labels
    # df = pd.read_csv("nih_labels.csv", index_col=0)

    # define torchvision transforms
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),      # Uncomment this line if flipping not done in preprocessing step
            transforms.Scale(224),
            # because scale doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Scale(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

    # create train/val data_loaders
    transformed_datasets = {
#         'train': CXR.ChexDataset(
        'train': ChexDataset(        
            path_to_images=PATH_TO_IMAGES,
            path_to_labels_csv=PATH_TO_LABELS,
            fold='train',
            transform=data_transforms['train'],
            sample=train_sampled
        ),
#         'val': CXR.ChexDataset(
        'val': ChexDataset(
            path_to_images=PATH_TO_IMAGES,
            path_to_labels_csv=PATH_TO_LABELS,
            fold='val',
            transform=data_transforms['val'],
            sample = val_sampled
        )}

    data_loaders = {
        'train': torch.utils.data.DataLoader(
            transformed_datasets['train'],
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=8),
        'val': torch.utils.data.DataLoader(
            transformed_datasets['val'],
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=8)}

    # please do not attempt to train without GPU as will take excessively long
    # if not use_gpu:
    #     raise ValueError("Error, requires GPU")
    model = models.densenet121(pretrained=True)
    num_features = model.classifier.in_features
    # add final layer with # outputs in same dimension of labels with sigmoid
    # activation
    model.classifier = nn.Sequential(nn.Linear(num_features, N_LABELS), nn.Sigmoid())

    # put model on GPU
    if (use_gpu):
        is_gpu_available = torch.cuda.is_available()
        if not is_gpu_available:
            raise ValueError("Error, Can't use GPU since hardware doesn't Support it, you idiot!")
        gpu_count = torch.cuda.device_count()
        print("Using GPU: Available GPU count:" + str(gpu_count))

        model = model.cuda()

    # define criterion, optimizer for training
    criterion = nn.BCELoss()
    optimizer = optim.SGD(
        filter(
            lambda p: p.requires_grad,
            model.parameters()),
        lr=learning_rate,
        momentum=0.9,
        weight_decay=WEIGHT_DECAY)
    dataset_sizes = {x: len(transformed_datasets[x]) for x in ['train', 'val']}

    # train model
    model, best_epoch = train_model(model, criterion, optimizer, learning_rate, num_epochs=NUM_EPOCHS,
                                    data_loaders=data_loaders, dataset_sizes=dataset_sizes, weight_decay=WEIGHT_DECAY)

    # get preds and AUCs on test fold
    preds,aucs = make_pred_multilabel(data_transforms, model, PATH_TO_IMAGES,PATH_TO_LABELS,test_sampled)

    return preds,aucs




Available GPU count:0


## Train-Validate

In [0]:
PATH_TO_IMAGES = "images/"
PATH_TO_LABELS = "/content/nih_labels.csv"
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.01
train_sampled=1
test_sampled=1
val_sampled=1
EPOCHS = 1
BATCH_SIZE = 16
preds, aucs = train_cnn(PATH_TO_IMAGES, PATH_TO_LABELS, LEARNING_RATE, WEIGHT_DECAY, False,train_sampled, val_sampled, test_sampled, EPOCHS, BATCH_SIZE)
# def         train_cnn(PATH_TO_IMAGES, PATH_TO_LABELS, learning_rate, WEIGHT_DECAY, use_gpu=False,train_sampled=0,val_sampled=0,test_sampled=0, NUM_EPOCHS=8, BATCH_SIZE=16):

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Epoch 1/20
----------
Epoch 1/20, phase:train


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


Epoch 1/20, phase:train, running_loss: 11.021450996398926, current_loss: 0.6888406872749329
Epoch 1/20, phase:train, running_loss: 21.254274368286133, current_loss: 0.6395514607429504
Epoch 1/20, phase:train, running_loss: 30.066996574401855, current_loss: 0.5507951378822327
Epoch 1/20, phase:train, running_loss: 37.152626514434814, current_loss: 0.44285187125205994
Epoch 1/20, phase:train, running_loss: 42.59288454055786, current_loss: 0.34001612663269043
Epoch 1/20, phase:train, running_loss: 46.58354067802429, current_loss: 0.24941600859165192
Epoch 1/20, phase:train, running_loss: 50.82808327674866, current_loss: 0.2652839124202728
Epoch 1/20, phase:train, running_loss: 54.05667018890381, current_loss: 0.20178668200969696
Epoch 1/20, phase:train, running_loss: 58.55576515197754, current_loss: 0.28119343519210815
Epoch 1/20, phase:train, running_loss: 61.66197729110718, current_loss: 0.19413825869560242
Epoch 1/20, phase:train, running_loss: 65.128586769104, current_loss: 0.21666309

In [1]:
print("Done")

Done
